# Basics of text processing

### Natural Language Processing and Information Extraction,  2023 WS
10/13/2023

Gábor Recski

## In this lecture
- Regular Expressions (SLP 2.1)
- Text segmentation and normalization (SLP 2.4)
   - sentence splitting and tokenization 
   - lemmatization, stemming, decompounding, morphology 
   
[SLP Ch. 2](https://web.stanford.edu/~jurafsky/slp3/2.pdf)

## Import dependencies

In [ ]:
import json
import re
from collections import Counter
import nltk
from nltk.tokenize import word_tokenize, sent_tokenize
import stanza

## Download models

In [ ]:
nltk.download('punkt')
nltk.download('stopwords')
stanza.download('en')
stanza.download('de')

## Regular expressions

- Pattern matching
- Substitution and grouping

### Pattern matching

We use a dataset of ca. 24K Wikipedia articles about movies after 2000 (created for the [GIR exercise](https://github.com/TUW-GIR/exercise-2023WS-template))

In [ ]:
!wget -nc -O data/wp_movie_data.jsonl https://owncloud.tuwien.ac.at/index.php/s/vYqIvzwulnSKJl0/download

In [ ]:
with open("data/wp_movie_data.jsonl") as f:
    movies = {item['title']: item['text'] for item in (json.loads(line) for line in f)}

In [ ]:
len(movies)

In [ ]:
def search_title(pattern, data, n=10):
    return sorted(title for title in data.keys() if re.match(pattern, title))[:n]

#### Which movies have the number 7 in their titles?

In [ ]:
def search_title(pattern, data):
    return sorted(title for title in data.keys() if re.search(pattern, title))[:]

In [ ]:
search_title('7', movies)

#### Limit it to those with 7 as a word

In [ ]:
search_title('(\s|^)7(\s|$)', movies)

In [ ]:
search_title('(\s|^)(7|[sS]even)(\s|$)', movies)

#### Let's try to find movies involving Aaron Sorkin

In [ ]:
def search_text(pattern, data, r=50):
    for title, text in data.items():
        match = re.search(pattern, text)
        if match is None:
            continue
        i, j = match.span()
        start = max(i-r, 0)
        end = i+r
        print(f"{title}\n\n...{text[start:end]}...\n\n")        


In [ ]:
search_text('Aaron Sorkin', movies)

#### Could we find all names in all texts?

In [ ]:
def count_patterns(pattern, data):
    return Counter(match for title, text in data.items() for match in re.findall(pattern, text)).most_common()

In [ ]:
name_pattern = '[A-Z][a-z]+(?: [A-Z][a-z]+)+'

In [ ]:
count_patterns(name_pattern, movies)

#### Let's reuse this pattern

In [ ]:
count_patterns('starring ' + name_pattern, movies)

In [ ]:
count_patterns(name_pattern+' franchise', movies)

In [ ]:
count_patterns('Academy Award for ' + name_pattern, movies)

### Substitution and groups

Regexes are not just for pattern matching, they are also a powerful tool for text manipulation.

In [ ]:
with open('data/tww_s1_e1.txt') as f:
    text = f.read()

In [ ]:
print(text)

Let's get the structure of this document, step by step

In [ ]:
match = re.search('(.*)\nACT ONE', text, re.S)
print(match)

In [ ]:
header = match.group(1).strip()
print(header)

In [ ]:
footer = re.search('THE END\n\* \* \*(.*)', text, re.S).group(1).strip()
print(footer)

We can do all this with a single regex

In [ ]:
header, body, footer = re.search('(.*)\n(ACT ONE.*THE END)\n\* \* \*(.*)', text, re.S).groups()

In [ ]:
print(header)

In [ ]:
print(footer)

In [ ]:
print(body)

Now let's get the scenes!

In [ ]:
SCENE_SEP_PATT = ("\n(?:CUT TO:|ACT [A-Z]*)")

In [ ]:
scenes = re.split(SCENE_SEP_PATT, body)

In [ ]:
len(scenes)

In [ ]:
print('\n\n'.join(f'{scene.strip()[:50]}...' for scene in scenes[:5]))

Now let's get the structure of the dialogue!

In [ ]:
print(scenes[2])

In [ ]:
LINE_PATT = "\n([A-Z. ]+)\n(.*?)\n"

In [ ]:
utterances = re.findall(LINE_PATT, scenes[0])

In [ ]:
utterances[:10]

In [ ]:
script = {
    "header": header,
    "scenes": [
        {"lines": [
            {
                "char": character,
                "text": text
            }
            for character, text in re.findall(LINE_PATT, scene)
        ]
        }
        for scene in re.split(SCENE_SEP_PATT, body)
        ],
    "footer": footer
}

In [ ]:
script

Let's use this data for something. Let's get a list of characters by frequency.

In [ ]:
Counter(line['char'] for scene in script['scenes'] for line in scene['lines']).most_common(10)

Regular expressions are surprisingly powerful. Also, with the right implementation, they are literally as fast as you can get. That's because they are equivalent to [finite state automata (FSAs)](https://en.wikipedia.org/wiki/Finite-state_machine). Actually, every regular expression is a [regular grammar](https://en.wikipedia.org/wiki/Regular_grammar) defining a [regular language](https://en.wikipedia.org/wiki/Regular_language).

![re_xkcd](media/re_xkcd.png)([XKCD #208](https://xkcd.com/208/))

## Text segmentation

### Splitting text into sentences

In [ ]:
text2 = "'Of course it's only because Tom isn't home,' said Mrs. Parsons vaguely."

#### Naive: split on `.`, `!`, `?`, etc.

In [ ]:
re.split('[.!?]', text2)

#### Better: use language-specific list of abbreviation words, collocations, etc.

In [ ]:
nltk.sent_tokenize(text2)

Custom lists of patterns are often necessary for special domains. 

In [ ]:
text3 = "An die Stelle der Landesgesetze vom 17. Jänner 1883, n.ö.L.G. u. V.Bl. Nr. 35, vom 26. Dezember 1890, n.ö.L.G. u. V.Bl. Nr. 48, vom 17. Juni 1920 n.ö.L.G. u. V.Bl. Nr. 547, vom 4. November 1920 n.ö.L.G. u. V.Bl. Nr. 808, und vom 9. Dezember 1927, L.G.Bl. für Wien Nr. 1 ex 1928, die, soweit dieses Gesetz nichts anderes bestimmt, zugleich ihre Wirksamkeit verlieren, hat die nachfolgende Bauordnung zu treten."

In [ ]:
print(text3)

In [ ]:
nltk.sent_tokenize(text3, language='german')

In [ ]:
nltk.sent_tokenize("17. Jänner", language='german')

In [ ]:
nltk.sent_tokenize("17. Januar", language='german')

###  Tokenization - splitting sentences into words

#### Naive approach: split on whitespace

In [ ]:
text2.split()

#### Better: separate punctuation marks

In [ ]:
re.findall('(\w+|[^\w\s]+)', text2)[:30]

#### Best: add some language-specific conventions:

In [ ]:
nltk.word_tokenize(text2)

In [ ]:
nltk.word_tokenize("O'Brian")

## Text normalization

#### What are the most common words in some sample of text?

In [ ]:
movie_sample = {title: text for i, (title, text) in enumerate(movies.items()) if i % 100 == 0}

In [ ]:
sorted(movie_sample.keys())

In [ ]:
words = [word for text in movie_sample.values() for word in nltk.word_tokenize(text)]

In [ ]:
words[:10]

In [ ]:
len(words)

In [ ]:
Counter(words).most_common(10)

Let's get rid of punctuation

In [ ]:
words = [word for word in words if re.match('\w', word)]

In [ ]:
len(words)

In [ ]:
Counter(words).most_common(10)

Filtering common function words is called __stopword removal__

In [ ]:
from nltk.corpus import stopwords
stopwords = set(stopwords.words('english'))
print(stopwords)

In [ ]:
words = [word for word in words if word.lower() not in stopwords]

In [ ]:
Counter(words).most_common(20)

### Lemmatization and stemming

Words like _say_, _says_, and _said_ are all different **word forms** of the same **lemma**. Grouping them together can be useful in many applications. 

**Stemming** is the reduction of words to a common prefix, using simple rules that only work some of the time:

In [ ]:
from nltk.stem import PorterStemmer
stemmer = PorterStemmer()

In [ ]:
for word in ('say', 'says', 'said'):
    print(stemmer.stem(word))

In [ ]:
for word in ('he', 'his', 'him'):
    print(stemmer.stem(word))

**Lemmatization** is the mapping of word forms to their lemma, using either a dictionary of word forms, a grammar of how words are formed (a **morphology**), or both.

In [ ]:
nlp = stanza.Pipeline('en', processors='tokenize,lemma,pos')

In [ ]:
text = movies["The Trial of the Chicago 7"]

In [ ]:
doc = nlp(text)

In [ ]:
for sentence in doc.sentences[:5]:
    for word in sentence.words:
        print(word.text + '\t' + word.lemma)
    print()

**QUESTION: Consider lemmas that could be reduced further, e.g. _historical_ or _protester_. Why aren't they?**

Now we can count lemmas

In [ ]:
Counter(
    word.lemma for sentence in doc.sentences for word in sentence.words
    if word.lemma.lower() not in stopwords and re.match('\w', word.lemma)).most_common(20)

The full analysis of how a word form is built from its lemma is known as **morphological analysis**

In [ ]:
for sentence in doc.sentences[:5]:
    for word in sentence.words:
        print('\t'.join([word.text, word.lemma, word.upos, word.feats if word.feats else '']))
    print()

A special case of lemmatization is **decompounding**, recognizing multiple lemmas in a word

In [ ]:
nlp('roller-coaster')

In [ ]:
nlp('wastebasket')

In [ ]:
nlp('anti-Vietnam')

In [ ]:
nlp('anti-fdgdhjfgjkf')

In [ ]:
nlp('underrated')

In [ ]:
nlp('overwhelmed')

For English you might say that this is good enough... but _some languages_ allow forming compounds on the fly...

In [ ]:
nlp_de = stanza.Pipeline('de', processors='tokenize,lemma,pos')

In [ ]:
nlp_de('Kraftfahrzeug-Haftpflichtversicherung')

In [ ]:
nlp_de('Nahrungsmittelunverträglichkeit')

In [ ]:
nlp_de('Rindfleischetikettierungsüberwachungsaufgabenübertragunsgesetz')

see also [https://de.wikipedia.org/wiki/Rindfleischetikettierungs%C3%BCberwachungsaufgaben%C3%BCbertragungsgesetz](https://de.wikipedia.org/wiki/Rindfleischetikettierungs%C3%BCberwachungsaufgaben%C3%BCbertragungsgesetz)

In [ ]:
nlp_de('Kassenidentifikationsnummer')

In [ ]:
nlp_de('Klimabonus')

There is no good generic solution and no standard tool. There are some unsupervised approaches like [SECOS](https://github.com/riedlma/SECOS) and [CharSplit](https://github.com/dtuggener/CharSplit), and there are also full-fledged morphological analyzers that might work, like [SMOR](https://www.cis.lmu.de/~schmid/tools/SMOR/) and its extensions [zmorge](https://pub.cl.uzh.ch/users/sennrich/zmorge/) and [SMORLemma](https://github.com/rsennrich/SMORLemma).

## Text preprocessing in NLP: best practices

Text preprocessing steps such as those above are critical components of most NLP applications. Very often they are also a main bottleneck.

**Preprocessing for segmentation and normalization should be a separate component in almost any NLP application**

When storing preprocessed text, the format should ensure **reproducibility** and it should be **platform-independent**. It should also be easy to **inspect** and allow for **version control**

### The CoNLL format

In [ ]:
from stanza.utils.conll import CoNLL

CoNLL.write_doc2conll(doc,"data/output.conllu")

In [ ]:
with open('data/output.conllu') as f:
    print(''.join(f.readlines()))

This format can be processed by several NLP libraries (stanza, spacy, nltk, etc.)

In [ ]:
!spacy convert data/output.conllu -c conllu data/

There is also a python library for reading them

In [ ]:
import conllu

In [ ]:
with open("data/output.conllu") as f:
    data = conllu.parse(f.read())

In [ ]:
data[0]

In [ ]:
data[0][4]

**For Milestone 1 of the Project exercise your team should gather the dataset(s) they are planning to use, perform standard preprocessing steps and INSPECT THE RESULTS to uncover potential issues that need to be handled. Finally, datasets should be stored in CoNLL-U format and pushed to the repository together with a short documentation of how the data was created.**